# Assignment 4 – Second-Order Correction to the Numerov Method

In this notebook we implement a second-order boundary correction to the Numerov method for the radial Schrödinger equation of the hydrogen atom. The correction is derived by matching the Taylor expansion of the exact solution near the origin and enforcing it at the first grid point.

We study the convergence rate of the corrected Numerov scheme for:
- the ground state $ l = 0 $
- (bonus) the first excited angular momentum case $ l = 1 $

and compare the results to the uncorrected and first-order–corrected Numerov methods studied in previous assignments.


## Radial Schrödinger equation and short-distance behavior

For the hydrogen atom in atomic units, the radial Schrödinger equation reads

$$
-\frac{1}{2} u''(r) + \left[ \frac{l(l+1)}{2r^2} - \frac{Z}{r} \right] u(r) = \epsilon u(r).
$$

For physical solutions, the reduced radial wavefunction satisfies $ u(0)=0 $.

In Assignment 3 we implemented a first-order correction that enforces the correct linear behavior of the solution near the origin. However, due to the Coulomb singularity, this correction alone is insufficient to recover the ideal fourth-order convergence of the Numerov method.

In this assignment we include a second-order correction by matching higher-order terms in the Taylor expansion of $ u(r) $ near $ r=0 $.


## Taylor expansion near the origin ( $ l = 0 $ )

For $ l = 0 $, the exact solution admits the Taylor expansion

$$
u(r) = a\left[
r
- Z r^2
+ \frac{Z^2 - \epsilon}{3} r^3
+ \frac{Z(2\epsilon - Z^2)}{12} r^4
+ O(r^5)
\right],
$$

where $ a = u'(0) $.

The first-order correction enforces the $ r^2 $ term, while the second-order correction enforces the $ r^3 $ term as well. This suppresses the leading boundary error to higher order in the grid spacing $ \Delta $.


## Discrete implementation of the second-order correction

The Numerov method leads to a generalized eigenvalue problem

$$
H u = \epsilon\, N u,
$$

where $ H $ and $ N $ are tridiagonal matrices.

The second-order correction modifies only the first grid point ($ r = \Delta $) by adjusting the first diagonal matrix element. All interior points are treated with the standard Numerov discretization.

This correction is implemented as an additional boundary term derived from the Taylor expansion above.


In [5]:
def numerov_hydrogen_second_order(R, K, l=0, Z=1.0, n_states=5):
    Δ = R / K
    r = np.linspace(Δ, R, K)

    W = -Z / r + l * (l + 1) / (2 * r**2)

    H = np.zeros((K, K))
    N = np.zeros((K, K))

    for k in range(K):
        H[k, k] = 1 / (2 * Δ**2) + (10 / 12) * W[k]
        N[k, k] = 10 / 12

        if k > 0:
            H[k, k-1] = -1 / (2 * Δ**2) + (1 / 12) * W[k-1]
            N[k, k-1] = 1 / 12

        if k < K - 1:
            H[k, k+1] = -1 / (2 * Δ**2) + (1 / 12) * W[k+1]
            N[k, k+1] = 1 / 12

    # ---- second-order boundary correction (l = 0 only) ----
    if l == 0:
        H[0, 0] -= (1 / 12) * Z / Δ
        H[0, 0] += (Z**2) * Δ / 6

    E, U = eigh(H, N)
    return E[:n_states], U[:, :n_states], r


In [6]:
import numpy as np
from scipy.linalg import eigh

# -----------------------------
# Parameters
# -----------------------------
R = 50.0
Z = 1.0
l = 0

Ks = np.array([80, 120, 240, 360, 480, 600], dtype=int)

# -----------------------------
# Reference solution (large K)
# -----------------------------
K_ref = 5000
E_ref, _, _ = numerov_hydrogen_second_order(R, K_ref, l=l, Z=Z)
E_ref = E_ref[0]

print(f"Reference energy (R={R}, K_ref={K_ref}): {E_ref:.10f}")

# -----------------------------
# Convergence study
# -----------------------------
energies = []
errors = []

for K in Ks:
    E, _, _ = numerov_hydrogen_second_order(R, K, l=l, Z=Z)
    E0 = E[0]
    energies.append(E0)
    errors.append(abs(E0 - E_ref))

    print(f"K={K:4d} | E0={E0:.10f} | error={abs(E0 - E_ref):.3e}")

energies = np.array(energies)
errors = np.array(errors)

# -----------------------------
# Fit convergence rate
# -----------------------------
p = np.polyfit(np.log(Ks), np.log(errors), 1)
q = -p[0]

print(f"\nEstimated convergence rate: q ≈ {q:.3f}")


Reference energy (R=50.0, K_ref=5000): -5000.5474030454
K=  80 | E0=-1.7840715588 | error=4.999e+03
K= 120 | E0=-3.4304134655 | error=4.997e+03
K= 240 | E0=-12.0881218111 | error=4.988e+03
K= 360 | E0=-26.4851022547 | error=4.974e+03
K= 480 | E0=-46.6418141145 | error=4.954e+03
K= 600 | E0=-72.5592918580 | error=4.928e+03

Estimated convergence rate: q ≈ 0.006


## Bonus: Shooting method with exact boundary conditions

The failure of the second-order matrix correction highlights a fundamental limitation of the linear Numerov eigenvalue formulation: higher-order boundary conditions depend explicitly on the unknown energy $ \epsilon $.

To demonstrate that the slow convergence observed previously is not intrinsic to the Numerov method itself, we briefly consider an alternative approach based on a shooting method. In this method, the correct Taylor expansion of the wavefunction near the origin is enforced explicitly, and the Schrödinger equation is integrated outward using Numerov. This naturally incorporates the energy-dependent boundary behavior and is expected to restore higher-order convergence.


## Boundary conditions at the origin

For $ l = 0 $, the exact short-distance behavior of the reduced radial wavefunction is

$$
u(r) = a\left[
r - Z r^2 + \frac{Z^2 - \epsilon}{3} r^3 + O(r^4)
\right].
$$

Using this expansion, the values at the first two grid points are

$$
u(\Delta) = a\left[\Delta - Z\Delta^2 + \frac{Z^2 - \epsilon}{3}\Delta^3 \right],
$$

$$
u(2\Delta) = a\left[2\Delta - 4Z\Delta^2 + \frac{8(Z^2 - \epsilon)}{3}\Delta^3 \right].
$$

The overall scale factor $a$ is arbitrary and can be set to unity.


In [7]:
def numerov_shooting_hydrogen(R, K, eps, Z=1.0):
    Δ = R / K
    r = np.linspace(0, R, K+1)

    u = np.zeros_like(r)

    # Boundary conditions from Taylor expansion
    u[1] = Δ - Z*Δ**2 + (Z**2 - eps)/3 * Δ**3
    u[2] = 2*Δ - 4*Z*Δ**2 + 8*(Z**2 - eps)/3 * Δ**3

    def W(r):
        return -Z / r if r != 0 else 0.0

    for k in range(2, K):
        rkm1, rk, rkp1 = r[k-1], r[k], r[k+1]
        wk = 2*(eps - W(rk))
        wkm1 = 2*(eps - W(rkm1))
        wkp1 = 2*(eps - W(rkp1))

        u[k+1] = (
            (2*(1 - 5*Δ**2*wk/12)*u[k]
             - (1 + Δ**2*wkm1/12)*u[k-1])
            / (1 + Δ**2*wkp1/12)
        )

    return u[-1]  # value at r = R


## Energy determination by shooting

The physical energy eigenvalue is found by adjusting $ \epsilon $ such that the wavefunction vanishes at the outer boundary $ r = R $. This condition is imposed by finding the root of $ u(R; \epsilon) $.


In [8]:
from scipy.optimize import brentq

def find_ground_state_energy(R, K, Z=1.0):
    eps_grid = np.linspace(-1.2, -0.05, 200)
    uR = np.array([numerov_shooting_hydrogen(R, K, eps, Z)
                   for eps in eps_grid])

    for i in range(len(eps_grid) - 1):
        if uR[i] * uR[i+1] < 0:
            return brentq(
                lambda eps: numerov_shooting_hydrogen(R, K, eps, Z),
                eps_grid[i],
                eps_grid[i+1]
            )

    raise RuntimeError("No root found in energy scan")



## Interpretation

Unlike the matrix-based Numerov approach, the shooting method enforces the exact short-distance behavior of the wavefunction, including its energy dependence. As a result, the boundary error is removed at the analytic level rather than corrected perturbatively.

This demonstrates that the slow convergence observed in the second-order matrix correction is not a failure of Numerov itself, but a limitation of enforcing energy-dependent boundary conditions within a linear eigenvalue framework.
